<a href="https://colab.research.google.com/github/Dongkyu1102/Data_final_project_Naver/blob/main/%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C_%EC%9D%91%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""워드클라우드_이동규_완.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1WuWAhFv4LwHP1UCmT2u6lUH_24PYM8Zj
"""

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import re
import random
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image

# 폰트 다운로드 런타임 재실행 필수 !!!!!!!!!!! 실행 및 재실행 안하면 시각화 타이틀 부분의 한글 깨짐 !!!!!!!!!!!
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

"""# 데이터 불러오기"""

PATH = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/토픽분류/'

PATH_2 = '/content/drive/MyDrive/final-project_kyu/프로젝트_시각화/'

# 크롤링 전체 데이터 불러오기
df = pd.read_csv(f"{PATH}최종.csv", encoding='utf-8-sig')

df = df.reset_index(drop=True)

df.info()

df_all_brands= {}
for brand_name in df['brand'].unique():
    df_all_brands[brand_name] = df.loc[df['brand'] == brand_name, 'content'].reset_index(drop=True)

df_all_brands

"""# 재구매 고객 추출"""

df_all_brands['라보에이치'] + df_all_brands['앙방']

# '라보에이치', '이지마인드', '닥터방기원', '보타믹스', '살림백서', '앙방', '비어멕', '트리플에스','TS샴푸', '반코르'

df_rebuy = df[df['재구매'] == 1]
df_rebuy

"""# 형태소분석(mecab) 설치"""

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

cd Mecab-ko-for-Google-Colab

!bash install_mecab-ko_on_colab_light_220429.sh

from konlpy.tag import Mecab
mecab = Mecab()

"""# 형태소분석(mecab) 전처리 by 주제/감정 완료 데이터"""

df_rebuy['brand'].unique()

# df_brands = {
#     '라보에이치': df_subject.loc[df_subject['brand'] == '라보에이치', 'content'].reset_index(drop=True),
#     '이지마인드': df_subject.loc[df_subject['brand'] == '이지마인드', 'content'].reset_index(drop=True),
#     '닥터방기원': df_subject.loc[df_subject['brand'] == '닥터방기원', 'content'].reset_index(drop=True),
#     '보타믹스': df_subject.loc[df_subject['brand'] == '보타믹스', 'content'].reset_index(drop=True),
#     '살림백서': df_subject.loc[df_subject['brand'] == '살림백서', 'content'].reset_index(drop=True),
#     '앙방': df_subject.loc[df_subject['brand'] == '앙방', 'content'].reset_index(drop=True),
#     '비어멕': df_subject.loc[df_subject['brand'] == '비어멕', 'content'].reset_index(drop=True),
#     '트리플에스': df_subject.loc[df_subject['brand'] == '트리플에스', 'content'].reset_index(drop=True),
#     'TS샴푸': df_subject.loc[df_subject['brand'] == 'TS샴푸', 'content'].reset_index(drop=True),
#     '반코르': df_subject.loc[df_subject['brand'] == '반코르', 'content'].reset_index(drop=True),
# }

df_brands= {}
for brand_name in df_rebuy['brand'].unique():
    df_brands[brand_name] = df_rebuy.loc[df_rebuy['brand'] == brand_name, 'content'].reset_index(drop=True)

df_brands['반코르']

# 라보에이치_content = []
# for i in range(df_brands['라보에이치'].shape[0]):
#     라보에이치_content.append(df_brands['라보에이치'][i])

rebuy_content = {}
for brand in df_rebuy['brand'].unique():
    brand_content = []
    for i in range(df_brands[brand].shape[0]):
        brand_content.append(df_brands[brand][i])
    rebuy_content[brand] = brand_content

len(rebuy_content['라보에이치'])

"""# 브랜드 별 키워드 분석(워드클라우드)"""

# '라보에이치', '이지마인드', '닥터방기원', '보타믹스', '살림백서', '앙방', '비어멕', '트리플에스','TS샴푸', '반코르'

rebuy_content['보타믹스']
rebuy_content['보타믹스'][:10]

mecab_lst = []
for text in rebuy_content['비어멕']:
    mecab_lst.append(mecab.nouns(text))

cnt_mecab = []
for i in mecab_lst:
    cnt_mecab += i

# 2글자 이상인 단어만 남기기
cnt_mecab = [word for word in cnt_mecab if len(word) >= 2]

# 지정된 단어를 제거하기
cnt_mecab = [word for word in cnt_mecab if word not in ['샴푸', '사용', '머리', '두피', '구매', '탈모', '배송', '감사', '생각',
                                                        '라보에' , '트리트', '기원', '이것', '제품', '통째', '이치', '이거', '부모',
                                                        '건데', '주문', '이번', '보타', '믹스', '트리플', '에스', '지디', '티에스', '구입', '코르' '의사',
                                                        '세정력', '거품', '머리카락', '만족'
                                                        ]]

for i in range(len(cnt_mecab)):
    if cnt_mecab[i] == '성비':
        cnt_mecab[i] = '가성비'
    elif cnt_mecab[i] == '가성':
        cnt_mecab[i] = '가성비'

for i in range(len(cnt_mecab)):
    if cnt_mecab[i] == '맥주':
        cnt_mecab[i] = '맥주효모'
    elif cnt_mecab[i] == '효모':
        cnt_mecab[i] = '맥주효모'


cnt_mecab

from collections import Counter
cnt = Counter(cnt_mecab)
cnt.most_common(20)

# 탑 30개의 단어만 워드클라우드를 위한  딕셔너리 형태로 재지정
cnt.most_common(50)

most_cnt = cnt.most_common(50)
most_cnt

for i, j in most_cnt:
    print(i, j)
    break

most_dic = {string : cnt for string, cnt in most_cnt}
most_dic

plt.rc('font', family='NanumGothic')

print(plt.rcParams['font.family']) # font
print(plt.rcParams['font.size']) # size

# 아래 폰드 파일은 공유폴더 아래주소에도 저장되어있음!!
# /content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/streamlit/NanumGothic.ttf

import PIL.Image

# 이미지 경로를 지정합니다.
img_path = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/ppt/워드클라우드/img.jpg'

# 이미지를 불러옵니다.
img = PIL.Image.open(img_path)

img_path_2 = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/ppt/워드클라우드/head.png'

# 이미지를 불러옵니다.
img_2 = PIL.Image.open(img_path_2)

img_array = np.array(img)
img_array_2 = np.array(img_2)

wordcloud = WordCloud(font_path='/content/drive/MyDrive/final-project_kyu/프로젝트_시각화/NanumGothic.ttf', mask=img_array_2, scale=2.0, max_font_size=250, background_color ='black', colormap='viridis',
                      width = 700, height = 700, random_state = 43).generate_from_frequencies(most_dic)

plt.figure(figsize = (6, 6)) # 최종 워드 클라우드 사이즈 지정
plt.imshow(wordcloud)
plt.title("비어멕_재구매_리뷰분석", size = 13)  # 타이틀 명 변경

plt.axis('off') # 그래프 축 제거

plt.show()

'지디', '구입',

# df_emotion_1 = df_emotion.dropna(subset=['가격(감정)', '머릿결(감정)', '배송/고객서비스(감정)', '세정력/거품(감정)', '탈모효과(감정)', '트러블/민감성(감정)', '향기(감정)'], how='all')
# df_emotion_1
# len(df_emotion.dropna(subset=['가격(감정)', '머릿결(감정)', '배송/고객서비스(감정)', '세정력/거품(감정)', '탈모효과(감정)', '트러블/민감성(감정)', '향기(감정)'], how='all'))

# 감정분류에서 모든 열이 동시에 결측값(NaN)인 것이 없으므로 빠진 게 없다.

# 감정분류 데이터에서 컬럼별로 나눠서 각 컬럼에서 1, 0을 나눈다.
# 예를 들어 어떠한 브랜드의 '가격' 컬럼에서 감정분류 1로 된 content를 추출했다치면 그 컬럼이 설령 배송에서는 0으로 라벨링 된 컬럼이라도
# 가격에 대한 컬럼으로 전체 데이터에서 추출할 것 이기에 워드클라우드하면 가격에 포커스 되서 시각화 될 것이라고 추론된다.
# 동사로 분석 필요함

"""# (브랜드 구별 x) 전체 워드클라우드"""

df_rebuy['content']

mecab_lst_all = []
for text in df_rebuy['content']:
    mecab_lst_all.append(mecab.nouns(text))

cnt_mecab_all = []
for i in mecab_lst_all:
    cnt_mecab_all += i

# 2글자 이상인 단어만 남기기
cnt_mecab_all = [word for word in cnt_mecab_all if len(word) >= 2]

# 지정된 단어를 제거하기
cnt_mecab_all = [word for word in cnt_mecab_all if word not in ['샴푸', '사용', '머리', '두피', '구매', '탈모', '배송', '감사', '생각',
                                                        '라보에' , '트리트', '기원', '이것', '제품', '통째', '이치', '이거', '부모',
                                                        '건데', '주문', '이번', '보타', '믹스', '트리플', '에스', '지디', '티에스', '구입', '코르' '의사', '머리카락']]

for i in range(len(cnt_mecab_all)):
    if cnt_mecab_all[i] == '성비':
        cnt_mecab_all[i] = '가성비'
    elif cnt_mecab_all[i] == '가성':
        cnt_mecab_all[i] = '가성비'

for i in range(len(cnt_mecab_all)):
    if cnt_mecab_all[i] == '맥주':
        cnt_mecab_all[i] = '맥주효모'
    elif cnt_mecab_all[i] == '효모':
        cnt_mecab_all[i] = '맥주효모'


cnt_mecab_all

from collections import Counter
cnt_all = Counter(cnt_mecab_all)
cnt_all.most_common(30)

# 탑 30개의 단어만 워드클라우드를 위한  딕셔너리 형태로 재지정
cnt_all.most_common(50)

most_cnt = cnt_all.most_common(50)
most_cnt

for i, j in most_cnt:
    print(i, j)
    break

most_dic = {string : cnt for string, cnt in most_cnt}
most_dic

img_array = np.array(img)

wordcloud = WordCloud(font_path='/content/drive/MyDrive/final-project_kyu/프로젝트_시각화/NanumGothic.ttf', mask=img_array_2, scale=2.0, max_font_size=250, background_color ='black', colormap='viridis',
                      width = 700, height = 700, random_state = 43).generate_from_frequencies(most_dic)

plt.figure(figsize = (6, 6)) # 최종 워드 클라우드 사이즈 지정
plt.imshow(wordcloud)
plt.title("재구매 고객_전체 탈모샴푸_키워드 분석", size = 13)  # 타이틀 명 변경

plt.axis('off') # 그래프 축 제거

plt.show()